#Initialisation

In [1]:
pip install geopy
import pandas as pd

In [2]:
df_main = pd.read_excel("Crime_Data.xlsx")

In [3]:
df_main.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [10]:
unib = df_main["Block"].unique().tolist()
# unib = len(df_main["Block"].unique())
# len(unib)/1000 
# unib1 = unib.tolist()
unib[1]

'020XX N LARAMIE AVE'

In [3]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import json

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

In [24]:
### use this when done separately in batch wise 
# f1 = open("bloc_Loc_detail.json")
# # uni_block_file = json.load(f1)
# uni_block = json.load(f1)
# f2 = open("Missing_bloc_Loc_detail.json")
# # missed_blocks_file = json.load(f2)
# missed_blocks = json.load(f2)

In [25]:
# will store the block with locations in uni_block and missing ones in missed_blocks
# comment the below 2 line while running the 2nd batch if you do it batch wise and not as whole
uni_block = {}
missed_blocks = {}
count = 0
for i in unib:
    # if count == 10000:  #---- un-comment to use it during batch wise extraction
    #     break
    if i not in uni_block.keys() and i not in missed_blocks.keys():
        count += 1
        # Input the street name
        street_name = preprocess_address(i)+", Chicago"
        
        # Perform geocoding
        # try:
        # location = geolocator.geocode(street_name)
        location = None
        for attempt in range(3):  # Retry up to 3 times
        
            try:
                time.sleep(1)
                location = geolocator.geocode(street_name)
                if location:
                    break
            # except GeocoderTimedOut:
            except:  
                time.sleep(1)  # Wait 1 second and retry
                # if attempt == 2:
                    
                    # missed_blocks[i] = street_name
                    # print("Missed block ", missed_blocks[i])
                print("missed_blocks", i)
                    
                continue
        
    
        # Check and print the results
        if location:
            uni_block[i] ={"latitude" :location.latitude ,"longitude": location.longitude, "address": location.address }
            print(f"Address: {location.address}")
            print(f"Latitude: {location.latitude}")
            print(f"Longitude: {location.longitude}") 
        else:
            uni_block[i] = "not found"
            missed_blocks[i] = street_name
            print("Missed block ", missed_blocks[i])
               
        






with open("bloc_Loc_detail.json", "w") as outfile1:
    json.dump(uni_block, outfile1)
with open("Missing_bloc_Loc_detail.json", "w") as outfile2:
    json.dump(missed_blocks, outfile2)


    



Missed block  4300 W 61 ST, Chicago
Missed block  4000 W 68 ST, Chicago
Address: 4300, South Kilpatrick Avenue, Sleepy Hollow, Garfield Ridge, Chicago, Lake Township, Cook County, Illinois, 60632, United States
Latitude: 41.81479995
Longitude: -87.74122471025828
Address: 4600 W 55th Street, West 55th Street, West Elsdon, Chicago, Lake Township, Cook County, Illinois, 60629, United States
Latitude: 41.7930081
Longitude: -87.7373882
Missed block  0300 W 27 ST, Chicago
Missed block  5300 S ELLSWORTH DR, Chicago
Address: Chicago, Burlington, & Quincy Railroad (CB&Q) Wooden Caboose # 14140, West Center Street, Douglas, Converse County, Wyoming, 82633, United States
Latitude: 42.7592689
Longitude: -105.3873214
Address: 6000, North Kirkwood Avenue, Sauganash, Forest Glen, Chicago, Jefferson Township, Cook County, Illinois, 60646, United States
Latitude: 41.99087
Longitude: -87.743339
Address: 2100, North New England Avenue, Beat 2512, Austin, Chicago, Jefferson Township, Cook County, Illinois

In [99]:
# unib[0]
print(count)
print(missed_blocks)
# print(uni_block)


48
{'062XX N MC CORMICK RD': '6200 N MC CORMICK RD, Chicago', '007XX E 103 ST': '0700 E 103 ST, Chicago'}


In [12]:
# import re

def preprocess_address(address):
    # Replace patterns like "XX" with "00"
    if 'X' in address[:5] and len(address) > 6:
        address_list = list(address) 
        for j in range(address[:6].count('X')):
            address_list[address_list.index('X')] = '0' 
        if address_list[0] == '0' and address_list[5] == ' ':
            address_list.pop(0)
        address = ''.join(address_list)
    return address


In [47]:
a = '062XX N MC CORMICK RD, Chicago'
print(preprocess_address(a))

06200 N MC CORMICK RD, Chicago


In [42]:
address_list = list(a)

In [43]:
address_list

['0',
 '6',
 '2',
 'X',
 'X',
 ' ',
 'N',
 ' ',
 'M',
 'C',
 ' ',
 'C',
 'O',
 'R',
 'M',
 'I',
 'C',
 'K',
 ' ',
 'R',
 'D',
 ',',
 ' ',
 'C',
 'h',
 'i',
 'c',
 'a',
 'g',
 'o']

In [76]:
location = geolocator.geocode("02500 N ELSTON AVE, Chicago")

In [77]:
location

Location(2500, North Elston Avenue, Logan Square, Chicago, West Chicago Township, Cook County, Illinois, 60647, United States, (41.9270653732821, -87.68054860958097, 0.0))

In [26]:
# use it when done in batches
# f1 = open("bloc_Loc_detail.json")
# uni_block_file = json.load(f1)

# f2 = open("Missing_bloc_Loc_detail.json")
# missed_blocks_file = json.load(f2)

In [27]:
len(uni_block_file.keys())

51020

In [28]:
len(missed_blocks_file.keys())

4736

In [23]:
for ii in missed_blocks_file.keys():
    print(ii)

062XX N MC CORMICK RD
007XX E 103 ST
046XX N LAKE SHORE DR NB
040XX W DIVISION ST
049XX S LAKE PARK AVE
003XX N DESPLAINES ST
050XX N DAMEN AVE
021XX S MICHIGAN AVE
064XX S MAPLEWOOD AVE
009XX N MASSASOIT AVE
022XX N MARMORA AVE
020XX S MICHIGAN AVE
047XX N SAWYER AVE
058XX S PRAIRIE AVE
001XX W RANDOLPH ST
077XX S PHILLIPS AVE
029XX W FLOURNOY ST
061XX S SACRAMENTO AVE
001XX E 114TH PL
014XX W RASCHER AVE
045XX W PALMER ST
076XX S MAPLEWOOD AVE
022XX N TRIPP AVE
001XX N LOTUS AVE
041XX S PULASKI RD
044XX S DR MARTIN LUTHER KING JR DR
050XX N KENMORE AVE
003XX E 89TH PL
013XX W 110TH ST
023XX W ADAMS ST
111XX S LANGLEY AVE
076XX S HAMILTON AVE
082XX S VERNON AVE
069XX S PRAIRIE AVE
025XX W ARGYLE ST
024XX E 77TH ST
044XX N CENTRAL AVE
058XX S PEORIA ST
061XX S LANGLEY AVE
044XX N BROADWAY
077XX S RIDGEWAY AVE
126XX S TORRENCE AVE
052XX W OHIO ST
007XX W 47TH PL
036XX W ROOSEVELT RD
050XX W SUPERIOR ST
047XX N ROCKWELL ST
041XX W CERMAK RD
020XX W EVERGREEN AVE
006XX W MONTROSE DR
073XX

In [140]:
df_block_ex = pd.DataFrame(uni_block_file)

In [146]:
# df_block_ex
df_block_ex.to_excel('uni_block_file.xlsx')

In [142]:
df_block_ex = df_block_ex.transpose()

In [56]:
df_block_ex.columns

Index(['Block', 'latitude', 'longitude', 'address'], dtype='object')

In [143]:
df_block_ex.reset_index(inplace = True)

In [144]:
df_block_ex = df_block_ex.rename(columns = {'index':'Block'}) 

In [44]:
df_block_ex.iloc[:,0]

074XX N ROGERS AVE            42.01619
020XX N LARAMIE AVE          41.917546
062XX N MC CORMICK RD        not found
019XX W BYRON ST             41.952503
086XX S COTTAGE GROVE AVE     41.73739
                               ...    
020XX E 86 ST                not found
070XX S CENTRAL PARK AV      41.765932
042XX W DRUMMOND PL          41.929071
030XX E 106 ST               not found
040XX W 70 ST                not found
Name: latitude, Length: 51020, dtype: object

In [53]:
df_block_ex = df_block_ex.drop('level_0', axis = 1)

In [58]:
df_block_ex.iloc[:10]

,Block,latitude,longitude,address
0,074XX N ROGERS AVE,42.01619,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge..."
1,020XX N LARAMIE AVE,41.917546,-87.756068,"2000, North Laramie Avenue, Beat 2515, Belmont..."
2,062XX N MC CORMICK RD,not found,not found,not found
3,019XX W BYRON ST,41.952503,-87.676505,"1900-1902, West Byron Street, Cuyler, North Ce..."
4,086XX S COTTAGE GROVE AVE,41.73739,-87.604954,"8600, South Cottage Grove Avenue, Dauphin Park..."
5,007XX E 103 ST,not found,not found,not found
6,002XX N Wells st,41.885851,-87.634043,"200, North Wells Street, Loop, Chicago, Cook C..."
7,023XX W JACKSON BLVD,41.877656,-87.684037,"2300, West Jackson Boulevard, Tri-Taylor, Near..."
8,073XX S JEFFERY BLVD,41.76239,-87.576328,"7300, South Jeffery Boulevard, South Shore, Ch..."
9,034XX N LAWNDALE AVE,41.942951,-87.720327,"3400, North Lawndale Avenue, Wacławowo, Avonda..."


In [60]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/323.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/323.6 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 92.2/323.6 kB 1.3 MB/s eta 0:00:01
   ------------------ --------------------- 153.6/323.6 kB 1.1 MB/s eta 0:00:01
   ------------------------ --------------- 194.6/323.6 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 323.6/323.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/16.2 MB 9.0 MB/s eta 0:00:02
    --------------------------------------- 0.4/16.2 MB 4.7 MB/s eta 0:00:04
   - -------------------------------------- 0.6/16.2 MB 4.7 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/16.2 MB 4.8 MB/s eta 0:00:04
   -- ------------------------------------- 1.1/16.2 MB 4.9 MB/s eta 0:00:0


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import geopandas as gpd
from shapely.geometry import Point

# Step 1: Load the Latitude and Longitude Data into a DataFrame
df_allblocks = pd.read_excel('ALL_blocks.xlsx')
data =  df_allblocks.copy()
data['geometry'] = data.apply(lambda row: Point(row['Longitude'], row['Latitude']) if (row['Longitude'] != 'not found' and row['Latitude'] != 'not found') else None, axis=1)

# Step 2: Convert to GeoDataFrame
geo_data = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:4326')  # EPSG:4326 is the standard for lat/lon

# Step 3: Load the District Shapefile
districts = gpd.read_file("C:\\Users\Hp\Desktop\Guvi Proj 2\policedistrict\PoliceDistrict.shp")  # Replace with the path to your shapefile
# Ensure the shapefile is in the same CRS as your GeoDataFrame
districts = districts.to_crs(geo_data.crs)

# Step 4: Perform Spatial Join
result = gpd.sjoin(geo_data, districts, how="left", predicate='intersects')

# Step 5: Add District Numbers to Original Data
data['district_number'] = result['DIST_NUM']  # Replace 'DIST_NUM' with the column name in shapefile

# print(data)


In [3]:
wards = gpd.read_file("C:\\Users\Hp\Desktop\Guvi Proj 2\wards\WARDS_2015.shp")  # Replace with the path to your shapefile

wards = wards.to_crs(geo_data.crs)

result = gpd.sjoin(geo_data, wards, how="left", predicate='intersects')


data['ward_number'] = result['WARD']  # Replace 'WARD' with the column name in your shapefile

# print(data)


In [4]:
community_area = gpd.read_file("C:\\Users\Hp\Desktop\Guvi Proj 2\community area\CommAreas.shp")  # Replace with the path to your shapefile

community_area = community_area.to_crs(geo_data.crs)

result = gpd.sjoin(geo_data, community_area, how="left", predicate='intersects')

data['Community_Area_number'] = result['AREA_NUMBE']  # Replace 'AREA_NUMBE' with the column name in your shapefile
data['Community_name'] = result['COMMUNITY']
# print(data)


In [85]:
# districts
# wards
community_area
# Neighbourhood

,AREA,PERIMETER,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN,geometry
0,0.0,0.0,0,0,35,DOUGLAS,35,4.600462e+07,31027.054510,"POLYGON ((1181573.25 1886828.04, 1181571.2 188..."
1,0.0,0.0,0,0,36,OAKLAND,36,1.691396e+07,19565.506153,"POLYGON ((1186289.356 1876750.733, 1186247.037..."
2,0.0,0.0,0,0,37,FULLER PARK,37,1.991670e+07,25339.089750,"POLYGON ((1176344.998 1871187.546, 1176346.5 1..."
3,0.0,0.0,0,0,38,GRAND BOULEVARD,38,4.849250e+07,28196.837157,"POLYGON ((1182322.043 1876674.731, 1182323.67 ..."
4,0.0,0.0,0,0,39,KENWOOD,39,2.907174e+07,23325.167906,"POLYGON ((1186289.356 1876750.733, 1186290.775..."
...,...,...,...,...,...,...,...,...,...,...
72,0.0,0.0,0,0,74,MOUNT GREENWOOD,74,7.558429e+07,48665.130539,"POLYGON ((1158152.274 1836518.25, 1158160.983 ..."
73,0.0,0.0,0,0,75,MORGAN PARK,75,9.187734e+07,46396.419362,"POLYGON ((1173045.882 1828593.191, 1172952.89 ..."
74,0.0,0.0,0,0,76,OHARE,76,3.718356e+08,173625.984660,"MULTIPOLYGON (((1119302.584 1938024.319, 11193..."
75,0.0,0.0,0,0,77,EDGEWATER,77,4.844999e+07,31004.830946,"POLYGON ((1168756.115 1942653.009, 1168756.116..."


In [5]:
data.head(20)

,Block,Latitude,Longitude,geometry,district_number,ward_number,Community_Area_number,Community_name
0,074XX N ROGERS AVE,42.01619,-87.675127,POINT (-87.675127 42.01619),24,49,1,ROGERS PARK
1,020XX N LARAMIE AVE,41.917838,-87.755969,POINT (-87.755968972 41.917838056),25,36,19,BELMONT CRAGIN
2,062XX N MC CORMICK RD,41.995219,-87.713355,POINT (-87.713354912 41.995219444),17,50,13,NORTH PARK
3,019XX W BYRON ST,41.952345,-87.677975,POINT (-87.677975059 41.952345086),19,47,5,NORTH CENTER
4,086XX S COTTAGE GROVE AVE,41.737751,-87.604856,POINT (-87.604855911 41.737750767),6,6,44,CHATHAM
5,007XX E 103 ST,not found,not found,None,NaN,NaN,NaN,NaN
6,002XX N Wells st,41.886018,-87.633938,POINT (-87.633937881 41.886018055),1,42,32,LOOP
7,023XX W JACKSON BLVD,41.877565,-87.684791,POINT (-87.68479102 41.877565108),12,27,28,NEAR WEST SIDE
8,073XX S JEFFERY BLVD,41.761919,-87.576209,POINT (-87.576209245 41.7619185),3,7,43,SOUTH SHORE
9,034XX N LAWNDALE AVE,41.943379,-87.719974,POINT (-87.7199738 41.943378528),17,30,21,AVONDALE


In [88]:
data.to_excel("Location_updated.xlsx")

In [150]:
data1 = data.copy()

In [153]:
merged_df = pd.merge(data1, df_main[['Block', 'Latitude','Longitude']], on='Block', how='left')#,suffixes = ('_file1', '_file2')) # add suffixes if needed

# Fill missing latitude values in file1 with values from file2
merged_df['latitude'] = merged_df['latitude'].fillna(merged_df['Latitude'])
merged_df['longitude'] = merged_df['longitude'].fillna(merged_df['Longitude'])

# Extract the updated DataFrame
updated_data1_df = merged_df[['Block', 'latitude','longitude','address']] 



In [138]:
# updated_data1_df = merged_df[['Block', 'latitude','longitude','address']] 
# data = updated_data1_df
data.head()

,Block,latitude,longitude,address,geometry,district_number,ward_number,Community_Area_number,Community_name
0,074XX N ROGERS AVE,-87.675127,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge...",POINT (-87.675127 -87.675127),NaN,NaN,NaN,NaN
1,074XX N ROGERS AVE,-87.675127,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge...",POINT (-87.675127 -87.675127),NaN,NaN,NaN,NaN
2,074XX N ROGERS AVE,-87.675127,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge...",POINT (-87.675127 -87.675127),NaN,NaN,NaN,NaN
3,074XX N ROGERS AVE,-87.675127,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge...",POINT (-87.675127 -87.675127),NaN,NaN,NaN,NaN
4,074XX N ROGERS AVE,-87.675127,-87.675127,"7400, North Rogers Avenue, Jarvis Square, Roge...",POINT (-87.675127 -87.675127),NaN,NaN,NaN,NaN


In [117]:
data1 = df_block_ex

In [92]:
df_main.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [154]:
updated_data1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549999 entries, 0 to 549998
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Block      549999 non-null  object 
 1   latitude   549401 non-null  float64
 2   longitude  549401 non-null  float64
 3   address    549999 non-null  object 
dtypes: float64(2), object(2)
memory usage: 16.8+ MB


In [151]:
# data1['latitude'] = data1['latitude'].apply(lambda x: None if x == "not found" else x)
data1['latitude'] = data1['latitude'].replace("not found", pd.NA) 
data1['longitude'] = data1['longitude'].replace("not found", pd.NA) 

In [8]:
data.info()
data.to_excel("full_loc.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549999 entries, 0 to 549998
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Block                  549999 non-null  object
 1   Latitude               549999 non-null  object
 2   Longitude              549999 non-null  object
 3   geometry               549399 non-null  object
 4   district_number        547450 non-null  object
 5   ward_number            547414 non-null  object
 6   Community_Area_number  547411 non-null  object
 7   Community_name         547411 non-null  object
dtypes: object(8)
memory usage: 33.6+ MB


In [6]:
data1[data1['address'] == 'not found'] #:- to verify that there is no 'not found' is in value

NameError: name 'data1' is not defined